# Pipeline Database Exploration

Inspect and analyze the saved pipeline configuration data stored in `pipelines.db`.

In [ ]:
from pathlib import Path
import sqlite3

DB_PATH = Path("pipelines.db").resolve()
print(f"Using database at {DB_PATH}")

def get_connection():
    return sqlite3.connect(DB_PATH)

with get_connection() as conn:
    tables = conn.execute(
        "SELECT name, type FROM sqlite_master WHERE type IN ('table','index','trigger') ORDER BY type, name"
    ).fetchall()

tables

In [ ]:
try:
    import pandas as pd
    from IPython.display import display
    print(f"pandas {pd.__version__} loaded")
except ImportError:
    pd = None
    display = None
    print("pandas is not installed; results will be shown as plain records.")

def fetch_records(query, params=None, frame=None):
    params = params or ()
    with get_connection() as conn:
        conn.row_factory = sqlite3.Row
        cur = conn.execute(query, params)
        rows = cur.fetchall()
    records = [dict(row) for row in rows]
    if (frame or (frame is None and pd is not None)) and pd is not None:
        df = pd.DataFrame.from_records(records)
        if display:
            display(df)
        return df
    for record in records:
        print(record)
    return records


In [ ]:
# View pipelines table
pipelines_df = fetch_records(
    """
    SELECT id,
           name,
           description,
           data_source,
           ensemble_enabled,
           ensemble_size,
           chain_of_verification,
           contextual_grounding,
           llm_as_judge,
           judge_model_id,
           created_at,
           updated_at
    FROM pipelines
    ORDER BY id
    """
)
pipelines_df

In [ ]:
# View pipeline steps with ordering
pipeline_steps_df = fetch_records(
    """
    SELECT p.id AS pipeline_id,
           p.name,
           pm.step_order,
           pm.model_id,
           pm.created_at
    FROM pipelines AS p
    LEFT JOIN pipeline_models AS pm
      ON pm.pipeline_id = p.id
    ORDER BY p.id, pm.step_order
    """
)
pipeline_steps_df

In [ ]:
if isinstance(pipelines_df, list):
    print("Aggregate preview skipped: pandas not available.")
elif pipelines_df is None or getattr(pipelines_df, "empty", True):
    print("No pipelines recorded yet.")
else:
    summary = (
        pipelines_df.assign(
            ensemble_enabled=pipelines_df["ensemble_enabled"].astype(bool),
            llm_as_judge=pipelines_df["llm_as_judge"].astype(bool)
        )
        .groupby(["ensemble_enabled", "llm_as_judge"], dropna=False)
        .size()
        .rename("pipeline_count")
        .reset_index()
    )
    if display:
        display(summary)
    summary
